# Creational patterns

The following are the properties of Creational patterns:
* They work on the basis of how objects can be created
* They isolate the details of object creation
* Code is independent of the type of object to be created

An example of a creational pattern is the **Singleton pattern**.

## Singleton Pattern

### Classical Singleton pattern

In [8]:
class Singleton(object):
    def __new__(cls, *args, **kwargs):
        if not hasattr(cls, 'instance'):
            cls.instance = object.__new__(cls, *args, **kwargs)
        return cls.instance


s = Singleton()
print("Object created", s)
s1 = Singleton()
print("Object created", s1)
s2 = Singleton()
print("Object created", s2)

Object created <__main__.Singleton object at 0x000001D38F08DDA0>
Object created <__main__.Singleton object at 0x000001D38F08DDA0>
Object created <__main__.Singleton object at 0x000001D38F08DDA0>


### Lazy instantiation in Singleton pattern

In [9]:
class Singleton:
    __instance = None
    def __init__(self):
        if not Singleton.__instance:
            print(" __init__ method called..")
        else:
            print("Instance already created:", self.getInstance())
    @classmethod
    def getInstance(cls):
        if not cls.__instance:
            cls.__instance = Singleton()
        return cls.__instance


s = Singleton() ## class initialized, but object not created
print("Object created", Singleton.getInstance()) # Object gets created here
s1 = Singleton() ## instance already created

 __init__ method called..
 __init__ method called..
Object created <__main__.Singleton object at 0x000001D38F677D30>
Instance already created: <__main__.Singleton object at 0x000001D38F677D30>


### The Monostate Singleton pattern

In [10]:
class Borg:
    __shared_state = {"1":"2"}
    def __init__(self):
        self.x = 1
        self.__dict__ = self.__shared_state
        pass

b = Borg()
b1 = Borg()
b.x = 4

print("Borg Object 'b': ", b)       ## b and b1 are distinct objects
print("Borg Object 'b1': ", b1)
print("Object State 'b':", b.__dict__)       ## b and b1 share same state
print("Object State 'b1':", b1.__dict__)

Borg Object 'b':  <__main__.Borg object at 0x000001D38F835CC0>
Borg Object 'b1':  <__main__.Borg object at 0x000001D38F8355C0>
Object State 'b': {'1': '2', 'x': 4}
Object State 'b1': {'1': '2', 'x': 4}


In [20]:
class Borg(object):
    _shared_state = {}
    def __new__(cls, *args, **kwargs):
        obj = super(Borg, cls).__new__(cls, *args, **kwargs)
        obj.__dict__ = cls._shared_state
        return obj

    
b = Borg()
b1 = Borg()
b.x = 4

print("Borg Object 'b': ", b)       ## b and b1 are distinct objects
print("Borg Object 'b1': ", b1)
print("Object State 'b':", b.__dict__)       ## b and b1 share same state
print("Object State 'b1':", b1.__dict__)

b1.x = 5
b1.y = 6
print("Object State 'b':", b.__dict__)       ## b and b1 share same state
print("Object State 'b1':", b1.__dict__)

print(id(b1._shared_state))
print(id(b._shared_state))

Borg Object 'b':  <__main__.Borg object at 0x000001D38F2DA8D0>
Borg Object 'b1':  <__main__.Borg object at 0x000001D38F2DA4E0>
Object State 'b': {'x': 4}
Object State 'b1': {'x': 4}
Object State 'b': {'x': 5, 'y': 6}
Object State 'b1': {'x': 5, 'y': 6}
2008149571336
2008149571336


### Singletons and metaclasses

In [21]:
class MetaSingleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, \
            cls).__call__(*args, **kwargs)
        return cls._instances[cls]
class Logger(metaclass=MetaSingleton):
    pass

logger1 = Logger()
logger2 = Logger()
print(logger1, logger2)

<__main__.Logger object at 0x000001D38F2DA2B0> <__main__.Logger object at 0x000001D38F2DA2B0>


### A real-world scenario – the Singleton pattern, part 1

Consider an example of a cloud service that involves multiple read and write
operations on the database. The complete cloud service is split across multiple services
that perform database operations. An action on the UI (web app) internally will call an
API, which eventually results in a DB operation.

It’s clear that the shared resource across different services is the database itself. So, if we
need to design the cloud service better, the following points must be taken care of:
* Consistency across operations in the database—one operation shouldn’t result in
conflicts with other operations
* Memory and CPU utilization should be optimal for the handling of multiple
operations on the databasem

In [27]:
import sqlite3

class MetaSingleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, \
            cls).__call__(*args, **kwargs)
        return cls._instances[cls]


class Database(metaclass=MetaSingleton):
    connection = None
    def connect(self):
        if self.connection is None:
            self.connection = sqlite3.connect("db.sqlite3")
            self.cursorobj = self.connection.cursor()
        return self.cursorobj


db1 = Database().connect()
db2 = Database().connect()
print ("Database Objects DB1", db1)
print ("Database Objects DB2", db2)

Database Objects DB1 <sqlite3.Cursor object at 0x000001D38FE57490>
Database Objects DB2 <sqlite3.Cursor object at 0x000001D38FE57490>


In [28]:
db3 = Database()
print ("Database Objects DB3", db3)
print(db3.connect())

Database Objects DB3 <__main__.Database object at 0x000001D38F4D7710>


通过阅读上面的代码，我们会发现以下几点。

* 1． 我们以MetaSingleton为名创建了一个元类。就像在上一节中解释的那样，Python的特殊方法\__call__可以通过元类创建单例。
* 2． 数据库类由MetaSingleton类装饰后，其行为就会表现为单例。因此，当数据库类被实例化时，它只创建一个对象。
* 3． 当Web应用程序对数据库执行某些操作时，它会多次实例化数据库类，但只创建一个对象。因为只有一个对象，所以对数据库的调用是同步的。此外，这样还能够节约系统资源，并且可以避免消耗过多的内存或CPU资源。

### A real-world scenario – the Singleton pattern, part 2

In [32]:
class HealthCheck:
    _instance = None
    
    def __new__(cls, *args, **kwargs):
        if not HealthCheck._instance:
            HealthCheck._instance = super(HealthCheck, \
            cls).__new__(cls, *args, **kwargs)
        return HealthCheck._instance
    
    def __init__(self):
        self._servers = []
        
    def addServer(self):
        self._servers.append("Server 1")
        self._servers.append("Server 2")
        self._servers.append("Server 3")
        self._servers.append("Server 4")
        
    def changeServer(self):
        self._servers.pop()
        self._servers.append("Server 5")


hc1 = HealthCheck()
hc2 = HealthCheck()

hc1.addServer()
print("Schedule health check for servers (1)..")
for i in range(4):
    print("Checking ", hc1._servers[i])

hc2.changeServer()
print("Schedule health check for servers (2)..")
for i in range(4):
    print("Checking ", hc2._servers[i])

Schedule health check for servers (1)..
Checking  Server 1
Checking  Server 2
Checking  Server 3
Checking  Server 4
Schedule health check for servers (2)..
Checking  Server 1
Checking  Server 2
Checking  Server 3
Checking  Server 5


Here are a few points that we should remember about Singletons:
    
* There are many real-world applications where we need to create only one object,
such as thread pools, caches, dialog boxes, registry settings, and so on. If we create
multiple instances for each of these applications, it will result in the overuse of
resources. Singletons work very well in such situations.
* Singleton; a time-tested and proven method of presenting a global point of access
without many downsides.
* Of course, there are a few downsides; Singletons can have an inadvertent impact
working with global variables or instantiating classes that are resource-intensive but
end up not utilizing them.

## The Factory Pattern

There are three variants of the Factory pattern:
* Simple Factory pattern: This allows interfaces to create objects without exposing
the object creation logic.
* Factory method pattern: This allows interfaces to create objects, but defers the
decision to the subclasses to determine the class for object creation.
* Abstract Factory pattern: An Abstract Factory is an interface to create related
objects without specifying/exposing their classes. The pattern provides objects of
another factory, which internally creates other objects.

### The Simple Factory pattern

<img src="simple_factory.png" />

In [34]:
from abc import ABCMeta, abstractmethod

class Animal(metaclass = ABCMeta):
    @abstractmethod
    def do_say(self):
        pass
    
class Dog(Animal):
    def do_say(self):
        print("Bhow Bhow!!")
        
class Cat(Animal):
    def do_say(self):
        print("Meow Meow!!")


## forest factory defined
class ForestFactory(object):
    def make_sound(self, object_type):
        return eval(object_type)().do_say()

    
## client code
if __name__ == '__main__':
    ff = ForestFactory()
    animal = input("Which animal should make_sound Dog or Cat?")
    ff.make_sound(animal)

Which animal should make_sound Dog or Cat?Dog
Bhow Bhow!!


### The Factory Method

<img src="factory.png" />

In [5]:
from abc import ABCMeta, abstractmethod

class Section(metaclass=ABCMeta):
    @abstractmethod
    def describe(self):
        pass

class PersonalSection(Section):
    def describe(self):
        print("Personal Section")
    
class AlbumSection(Section):
    def describe(self):
        print("Album Section")
        
class PatentSection(Section):
    def describe(self):
        print("Patent Section")
        
class PublicationSection(Section):
    def describe(self):
        print("Publication Section")
        
        
class Profile(metaclass=ABCMeta):
    def __init__(self):
        self.sections = []
        self.createProfile()
        
    @abstractmethod
    def createProfile(self):
        pass
    
    def getSections(self):
        return self.sections
    
    def addSections(self, section):
        self.sections.append(section)


class linkedin(Profile):
    def createProfile(self):
        self.addSections(PersonalSection())
        self.addSections(PatentSection())
        self.addSections(PublicationSection())


class facebook(Profile):
    def createProfile(self):
        self.addSections(PersonalSection())
        self.addSections(AlbumSection())
        

if __name__ == '__main__':
    profile_type = input("Which Profile you'd like to create? [LinkedIn or FaceBook]")
    profile = eval(profile_type.lower())()
    print("Creating Profile..", type(profile).__name__)
    print("Profile has sections --", profile.getSections())

Which Profile you'd like to create? [LinkedIn or FaceBook]linkedin
Creating Profile.. linkedin
Profile has sections -- [<__main__.PersonalSection object at 0x000001E9DF095240>, <__main__.PatentSection object at 0x000001E9DF3B48D0>, <__main__.PublicationSection object at 0x000001E9DF3B46A0>]


# Structural patterns

The following are the properties of Structural patterns:
* They design the structure of objects and classes so that they can compose to achieve
larger results
* The focus is on simplifying the structure and identifying the relationship between
classes and objects
* They focus on class inheritance and composition

An example of a behavior pattern is the **Adapter pattern**.

# Behavioral patterns

The following are the properties of Behavioral patterns:
* They are concerned with the interaction among objects and responsibility of objects
* Objects should be able to interact and still be loosely coupled

An example of a behavioral pattern is the **Observer pattern**.